In [1]:
import pandas as pd
import pandas_datareader as web
import requests
import numpy as np

In [2]:
from pandas_datareader._utils import RemoteDataError

def get_ff_factors(START_DATE):
    try:
        df_five_factor = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench',
                                        start=START_DATE)[0]
        df_five_factor = df_five_factor.div(100)
        df_five_factor.index = df_five_factor.index.strftime('%Y-%m-%d')
        df_five_factor.index.name = 'Date'
        df_five_factor.columns = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
        df_five_factor.index = pd.to_datetime(df_five_factor.index)

        momentum_factor = web.DataReader('F-F_Momentum_Factor_daily', 'famafrench', START_DATE  )[0]

        combined_factors = df_five_factor.join(momentum_factor)

        return combined_factors
    except RemoteDataError as e:
        print(f"Failed to fetch data: {e}")
        return None

# Example usage
ff = get_ff_factors(START_DATE='1990-01-01')

C:\Users\micha\AppData\Local\Temp\ipykernel_1272\2402337028.py:5: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_five_factor = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench',
C:\Users\micha\AppData\Local\Temp\ipykernel_1272\2402337028.py:13: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  momentum_factor = web.DataReader('F-F_Momentum_Factor_daily', 'famafrench', START_DATE  )[0]


In [3]:
ff = ff.shift(1).dropna()

In [4]:
ff

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1990-01-03,0.0144,-0.0067,-0.0007,0.0019,-0.0044,0.00026,-1.09
1990-01-04,-0.0006,0.0072,-0.0026,0.0014,-0.0043,0.00026,-0.31
1990-01-05,-0.0071,0.0044,-0.0022,-0.0006,-0.0003,0.00026,-0.38
1990-01-08,-0.0085,0.0074,-0.0021,-0.0001,-0.0012,0.00026,-0.33
1990-01-09,0.0030,-0.0040,-0.0026,0.0006,0.0031,0.00026,0.18
...,...,...,...,...,...,...,...
2024-12-24,0.0061,-0.0085,-0.0019,0.0028,0.0066,0.00017,-0.08
2024-12-26,0.0111,-0.0012,-0.0005,-0.0013,-0.0037,0.00017,0.67
2024-12-27,0.0002,0.0109,-0.0019,-0.0044,0.0035,0.00017,0.01


In [6]:
import pandas as pd
import requests
from pathlib import Path
import sys, os
import json

frequency_mapping = {
        "GDP": "Q",  # Quarterly
        "CPIAUCSL": "M",  # Monthly
        "UNRATE": "M",  # Monthly
        "BAMLH0A0HYM2": "D",  # Daily
        "SP500": "D",  # Daily
        "DGS10": "D",  # Daily
        "DGS5": "D",  # Daily
        "DGS1MO": "D",  # Daily
        "DGS3MO": "D",  # Daily
        "DGS6MO": "D",  # Daily
        "DGS1": "D",  # Daily
        "DGS2": "D",  # Daily
        "DGS3": "D",  # Daily
        "DGS7": "D",  # Daily
        "DGS20": "D",  # Daily
        "DGS30": "D",  # Daily
        "DCOILWTICO": "D",  # Daily
        "DCOILBRENTEU": "D",  # Daily
        "DAAA": "D",  # Daily
        "DBAA": "D",  # Daily
        "AAA": "D",  # Daily
        "BAA": "D",  # Daily
        "T10Y2Y": "D",  # Daily
        "T10Y3M": "D",  # Daily
        "T10YFF": "D",  # Daily
        "T10YIE": "D",  # Daily
        "T5YFFM": "D",  # Daily
        "CBBTCUSD": "D",  # Daily
    }

def get_matrix(ids=frequency_mapping.keys()):
    """ Retrieve multiple Fred data series' and return a dataframe. """
    parent = Path('').resolve()#.parent
    SECRET_FP = os.path.join(parent, "secrets.json")
    with open(SECRET_FP, 'r') as file:
        secrets = json.load(file)
    API_KEY = secrets['fred_api_key']

    merged_data = None

    for series_id in ids:
        params = {
            "series_id": series_id,
            "api_key": 'ae9eac413fee6bc7cbe0747b78d0b32c',
            "file_type": "json",
        }
        FRED_URL = "https://api.stlouisfed.org/fred/series/observations"
        response = requests.get(FRED_URL, params=params)

        if response.status_code == 200:
            data = response.json()
            observations = data["observations"]
            df = pd.DataFrame(observations)
            df["value"] = pd.to_numeric(df["value"], errors="coerce")
            df = df[["date", "value"]]
            df.rename(columns={"value": series_id}, inplace=True)
            df["date"] = pd.to_datetime(df["date"])
            df.set_index("date", inplace=True)

            # Adjust for look-ahead bias
            frequency = frequency_mapping.get(series_id, "D")  # Default to daily if not specified
            if frequency == "QE":
                df = df.shift(1, freq='QE')
            elif frequency == "ME":
                df = df.shift(1, freq='ME')
            else:
                df = df.shift(1)

            if merged_data is None:
                merged_data = df
            else:
                merged_data = merged_data.merge(
                    df, how="outer", left_index=True, right_index=True
                )

        else:
            print(f"Failed to fetch data for {series_id}. Status code: {response.status_code}")
            pass

    merged_data.index = pd.to_datetime(merged_data.index)

    return merged_data

# Example usage
macro = get_matrix()

In [7]:
macro

,GDP,CPIAUCSL,UNRATE,BAMLH0A0HYM2,SP500,DGS10,DGS5,DGS1MO,DGS3MO,DGS6MO,...,DAAA,DBAA,AAA,BAA,T10Y2Y,T10Y3M,T10YFF,T10YIE,T5YFFM,CBBTCUSD
date,,,,,,,,,,,,,,,,,,,,,
1919-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.35,7.12,NaN,NaN,NaN,NaN,NaN,NaN
1919-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.35,7.20,NaN,NaN,NaN,NaN,NaN,NaN
1919-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.39,7.15,NaN,NaN,NaN,NaN,NaN,NaN
1919-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.44,7.23,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96475.82
2025-02-10,NaN,NaN,NaN,2.67,6025.99,4.49,4.34,4.37,4.35,4.30,...,5.33,5.94,NaN,NaN,0.20,0.14,0.16,2.42,NaN,97444.41
2025-02-11,NaN,NaN,NaN,2.66,6066.44,4.51,4.34,4.38,4.35,4.31,...,5.35,5.96,NaN,NaN,0.23,0.16,0.18,2.44,NaN,95774.08


In [8]:
import datetime
import yfinance as yf
def get_price_matrix(tickers, start_date='1990-01-01', end_date=datetime.datetime.now().strftime('%Y-%m-%d')):
    return yf.download(tickers, start_date, end_date)['Close']

px = get_price_matrix(tickers = ['IVV', 'AGG', 'TLT',
                            'IWM', 'IWN', 'IWO',
                            'IWB', 'IWF', 'IWD',
                            'IWR', 'IWP', 'IWS',  
                            ])

[*********************100%***********************]  12 of 12 completed


In [9]:
px

Ticker,AGG,IVV,IWB,IWD,IWF,IWM,IWN,IWO,IWP,IWR,IWS,TLT
Date,,,,,,,,,,,,
2000-05-19,NaN,89.948845,48.677162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05-22,NaN,89.389389,48.533878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05-23,NaN,88.030807,47.715103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05-24,NaN,89.349434,48.124489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05-25,NaN,88.530334,47.766254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-06,97.769997,609.400024,334.470001,194.410004,412.880005,228.649994,169.059998,299.529999,137.850006,92.589996,133.580002,89.849998
2025-02-07,97.459999,603.799988,331.559998,193.250000,408.109985,226.000000,167.089996,296.200012,137.619995,92.160004,132.869995,89.269997
2025-02-10,97.489998,607.750000,333.709991,193.690002,412.260010,227.020004,167.619995,297.570007,138.919998,92.459999,132.889999,89.000000


In [10]:
rets = np.log(px / px.shift(1))
rets

Ticker,AGG,IVV,IWB,IWD,IWF,IWM,IWN,IWO,IWP,IWR,IWS,TLT
Date,,,,,,,,,,,,
2000-05-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05-22,NaN,-0.006239,-0.002948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05-23,NaN,-0.015315,-0.017014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05-24,NaN,0.014868,0.008543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05-25,NaN,-0.009210,-0.007472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-06,-0.001022,0.003666,0.003144,-0.000051,0.005781,-0.003275,-0.000414,-0.005593,0.005310,0.000432,-0.001571,-0.000445
2025-02-07,-0.003176,-0.009232,-0.008738,-0.005985,-0.011620,-0.011657,-0.011721,-0.011180,-0.001670,-0.004655,-0.005329,-0.006476
2025-02-10,0.000308,0.006521,0.006464,0.002274,0.010118,0.004503,0.003167,0.004615,0.009402,0.003250,0.000151,-0.003029


In [11]:
fwd_rets = rets.shift(-252).rolling(window=252).sum().dropna()
fwd_rets.columns = [f'{col}_fwd_1y' for col in fwd_rets.columns]
fwd_rets

,AGG_fwd_1y,IVV_fwd_1y,IWB_fwd_1y,IWD_fwd_1y,IWF_fwd_1y,IWM_fwd_1y,IWN_fwd_1y,IWO_fwd_1y,IWP_fwd_1y,IWR_fwd_1y,IWS_fwd_1y,TLT_fwd_1y
Date,,,,,,,,,,,,
2003-09-29,0.035386,0.122678,0.121792,0.181822,0.063541,0.159006,0.215951,0.100138,0.110885,0.175173,0.216991,0.070241
2003-09-30,0.031673,0.132131,0.129967,0.189818,0.069349,0.172108,0.232623,0.112182,0.129623,0.183828,0.227663,0.049939
2003-10-01,0.030372,0.127308,0.127389,0.182938,0.064414,0.170367,0.224103,0.111734,0.125266,0.183871,0.227319,0.048728
2003-10-02,0.032225,0.125145,0.125704,0.182390,0.067073,0.166420,0.222255,0.111739,0.123031,0.185115,0.220934,0.054794
2003-10-03,0.039374,0.116656,0.118481,0.173878,0.053975,0.149806,0.205983,0.087824,0.103697,0.168530,0.208849,0.076022
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-05,0.038548,0.221383,0.224033,0.181655,0.256420,0.186704,0.164612,0.208436,0.272223,0.198398,0.172811,-0.005932
2024-02-06,0.030164,0.209448,0.211900,0.169696,0.243617,0.166074,0.145454,0.186787,0.259801,0.185973,0.160668,-0.022134
2024-02-07,0.032511,0.207883,0.209598,0.169327,0.240999,0.172903,0.151957,0.191643,0.259121,0.183050,0.156822,-0.020312


In [12]:
data = ff.merge(macro, left_index=True, right_index=True).merge(rets, left_index=True, right_index=True).merge(fwd_rets, left_index=True, right_index=True).ffill().dropna()

In [13]:
data

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom,GDP,CPIAUCSL,UNRATE,...,IWB_fwd_1y,IWD_fwd_1y,IWF_fwd_1y,IWM_fwd_1y,IWN_fwd_1y,IWO_fwd_1y,IWP_fwd_1y,IWR_fwd_1y,IWS_fwd_1y,TLT_fwd_1y
2015-02-18,0.0017,-0.0002,-0.0006,-0.0018,-0.0015,0.00000,0.01,17804.228,236.983,5.8,...,-0.083374,-0.111650,-0.057704,-0.185202,-0.170019,-0.202937,-0.135692,-0.141012,-0.147244,0.055318
2015-02-19,0.0003,0.0026,-0.0081,0.0029,-0.0018,0.00000,0.66,17804.228,236.983,5.8,...,-0.082675,-0.109697,-0.057994,-0.180294,-0.165311,-0.198415,-0.133667,-0.139414,-0.145758,0.062795
2015-02-20,-0.0001,0.0021,-0.0037,-0.0005,-0.0037,0.00000,-0.57,17804.228,236.983,5.8,...,-0.074823,-0.099912,-0.051720,-0.171905,-0.156095,-0.188794,-0.128784,-0.131246,-0.134956,0.059331
2015-02-23,0.0061,-0.0044,-0.0027,-0.0007,-0.0010,0.00000,0.56,17804.228,236.983,5.8,...,-0.086082,-0.111689,-0.063084,-0.182084,-0.166775,-0.199265,-0.135134,-0.140702,-0.147647,0.051977
2015-02-24,-0.0008,0.0004,-0.0037,0.0038,-0.0022,0.00000,0.93,17804.228,236.983,5.8,...,-0.085089,-0.111600,-0.059143,-0.173797,-0.160616,-0.188207,-0.129154,-0.134896,-0.141057,0.036957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-05,0.0099,-0.0157,-0.0051,-0.0016,-0.0140,0.00021,1.66,25805.791,309.794,3.7,...,0.224033,0.181655,0.256420,0.186704,0.164612,0.208436,0.272223,0.198398,0.172811,-0.005932
2024-02-06,-0.0042,-0.0094,-0.0066,0.0010,-0.0015,0.00021,1.23,25805.791,309.794,3.7,...,0.211900,0.169696,0.243617,0.166074,0.145454,0.186787,0.259801,0.185973,0.160668,-0.022134
2024-02-07,0.0027,0.0077,-0.0049,-0.0075,0.0059,0.00021,-1.41,25805.791,309.794,3.7,...,0.209598,0.169327,0.240999,0.172903,0.151957,0.191643,0.259121,0.183050,0.156822,-0.020312
2024-02-08,0.0077,-0.0102,-0.0062,0.0062,-0.0081,0.00021,0.93,25805.791,309.794,3.7,...,0.207706,0.170053,0.236311,0.152022,0.140669,0.161118,0.237277,0.172130,0.150236,-0.020906


In [14]:
data.to_csv('data.csv')

In [15]:
[c for c in data.columns if '_fwd_' in c]

['AGG_fwd_1y',
 'IVV_fwd_1y',
 'IWB_fwd_1y',
 'IWD_fwd_1y',
 'IWF_fwd_1y',
 'IWM_fwd_1y',
 'IWN_fwd_1y',
 'IWO_fwd_1y',
 'IWP_fwd_1y',
 'IWR_fwd_1y',
 'IWS_fwd_1y',
 'TLT_fwd_1y']